In [1]:
# for notbook shortcut command 

In [2]:
%matplotlib inline
import sys, os, pdb, warnings, torch, time
sys.path.insert(0, './core/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
from itertools import product
from copy import deepcopy
from torch.utils.data import DataLoader, Dataset

from minilib import *
from utmLib import utils
from utmLib.clses import Logger

np.set_printoptions(suppress=True, linewidth=120, precision=4)
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 1000)
plt.rcParams["figure.figsize"] = [10,2]

# this is mainly for papermill parameter detection
try:
    dummy_x89757 = data_name
except:
    data_name = 'parkinson'
    log_file = 'results/exp.log'
    G_delta = 0.5
    testing = 1

logger = Logger(log_file)
res_log = Logger(log_file + '.res', with_time = False)


# Utility functions 

In [3]:
def visualize_imgs(img_array, rows = 2, cols = 8, selected = None, label_array = None):
    if img_array[0].size == 196:
        process = lambda x:x.reshape(14,14)
    else:
        process = lambda x:x.reshape(28,28)
    
    # random select some examples for display if not specified
    if selected is None:
        assert(img_array.shape[0] >= rows * cols)
        selected = np.random.choice(img_array.shape[0], rows * cols, replace = False)
    else:
        assert(selected.size >= rows * cols)
    
    k = 1
    labels = []
    fid = plt.figure()
    
    for i in range(rows):
        for j in range(cols):
            plt.subplot(rows, cols, k)
            plt.imshow(process(img_array[selected[k-1]]), cmap='gray' )
            if label_array is not None:
                labels.append(label_array[selected[k-1]])
            k += 1
            plt.axis('off')
    
    if len(labels):
        print(labels)
    plt.show()

In [4]:
def adversial_step(masses, delta = 0.5):
    
    def get_w(alpha):
        w = np.exp( -masses / alpha )
        w = w / np.sum(w) * N
        return w
    
    def valid(alpha):
        w = get_w(alpha)
        return np.log(np.power(w,w)).sum() <= M
    
    # use the idea of binary search, time complexity O(N * lgN)
    N = masses.size
    M = N * delta
    l = 0.01
    r = 2 ** 10
    
    # need to gurantee that r is big enough 
    while not valid(r):
        r = r * 2
    
    # binary search a valid alpha in range [l,r] 
    while r - l > 1e-2:
        m = (l+r) / 2

        if valid(m):
            r = m 
        else:
            l = m
            
    return get_w(r)


In [5]:
def py_learning_step(step, R, data_loader, model, optimizer, scheduler, max_epoch):
    model.train()
    R = R.astype('f4').reshape(-1, 1)
    R_tensor = torch.from_numpy( R ).to(model.device)
    
    for e in range(max_epoch):
        acc_loss = 0
        total = 0

        for X, Y, _i in data_loader:
            X = X.to(model.device)
            Y = Y.to(model.device)
            
            optimizer.zero_grad()
            out = model.forward(X)
            loss = model.loss(out, Y, R = R_tensor[_i])
            loss.backward()
            optimizer.step()
            
            acc_loss += loss.item()
            total += X.shape[0]
        
        # step is global variable
        scheduler.step()
        if VERBOSE >= 1:
            print(f'Step {step}, Learning epoch {e}, avg loss: {acc_loss/total}', end = '\r')
    return acc_loss/total
    

In [6]:
def mg_learning_step(R, data, **kwargs):
    # learn a gaussian distribution from weighted data
    N, D = data.shape 
    mg = MultivariateGaussain()
    R = R.reshape(N, 1)
    mg.mu = np.mean( R * data, axis = 0)
    
    mat = data - mg.mu.reshape(1, D)
    mat2 = mat.copy()
    mat2 = mat2 * R / N
    S2 = mat.T @ mat2
    
    mg.S = S2
    return mg

def mixmg_learning_step(R, data, n_comp, **kwargs):
    step = kwargs['step']
    total = kwargs['total']
    adv = kwargs['adv']
    wrapper = kwargs['wrapper']
    
    if wrapper[0] is None:
        wrapper[0] = MixMGLearner(max_iter = 0, n_components = n_comp,
                                  reg_covar = 1e-4).fit(data)
    
    mixmg_object = wrapper[0]
    
    if step == 0:
        niter = 50
    else:
        niter = 1
        
    for _ in range(niter):
        mixmg_object._estep()
        mixmg_object._mstep(data, R)
    
    return mixmg_object.get_model()

In [7]:
class MyData(Dataset):
    def __init__(self, data, _x, _y):
        self.X = data[:, _x]
        self.Y = data[:, _y]
        self.total = data.shape[0]
    
    def __len__(self):
        return self.total
    
    def __getitem__(self, ind):
        return self.X[ind], self.Y[ind], ind

def five_number_statistic(logmass):
    p25, median, p75 = np.percentile(logmass, [25,50,75])
    average = np.mean(logmass)
    std = np.std(logmass)
    ret = (p25, median, p75, average, std)
    return list(np.round(ret, 4))

# training and experiment helpers 

In [8]:
def train_model(train_data, val_data, train_conf, model_conf, lr, wd, device):
    model_conf.device = device
    xid = model_conf.xid
    yid = model_conf.yid
    num_parents = model_conf.num_parents
    structure = model_conf.structure
    
    batch_size = train_conf.batch_size
    init_epoch = train_conf.init_epoch
    n_epoch = train_conf.n_epoch
    n_step = train_conf.n_step
    adv = train_conf.adversial
    delta = train_conf.delta
    n_pretrain = train_conf.n_pretrain
    pre_epochs = train_conf.pre_epochs

    if VERBOSE >= 2:
        model_ids = [id(m) for m in train_conf.model_objs]
        print('Model IDs: {}'.format(model_ids) )
    
    # init
    px_container = [None]  # for mixmg px part use only in concurrent environment
    R = np.ones( shape = (train_data.shape[0], ) )
    total_epochs = init_epoch + n_epoch + n_step - 1 
    data_loader = DataLoader( MyData(train_data, xid, yid) , batch_size = batch_size)

    # pretraining, reduce the effect of random init of neural network 
    best_score = np.inf
    best_nn = None
    for i in range(n_pretrain):
        # model = PGNN( len(xid) , num_parents, model_conf)
        model = deepcopy(train_conf.model_objs[i])
        model.moveto(device)
        model.loss = partial(model.weighted_loss_func, G = structure)
        optimizer = torch.optim.Adam(model.parameters(), weight_decay = wd )
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr,
                                anneal_strategy='cos', pct_start=0.2,
                                epochs= total_epochs, steps_per_epoch = 1, verbose = False)
        model.opt = optimizer
        model.sch = scheduler
        score = py_learning_step('Init', R, data_loader, model, optimizer, 
                     scheduler, max_epoch = pre_epochs)

        if VERBOSE >= 2:
            print(f'In trial #{i}, with loss {score}')
        
        if score < best_score:
            if VERBOSE >= 2:
                print(f'update the best model as trial #{i}')
            best_nn = model
            best_score = score

    # revert to the best model 
    model = best_nn
    optimizer = model.opt
    scheduler = model.sch

    # finish train to init_epoches 
    py_learning_step('Init', R, data_loader, model, optimizer, 
                     scheduler, max_epoch = max(init_epoch - pre_epochs, 0) )
    
    # create the p(y|x) model object
    py = NeuralNetCondMG()
    py.nn = model
    py.gbn = GBN(structure).fit(train_data[:, yid], var_thresh = 1e-2)
    
    # begining of (adversial robust) training 
    best_score = -np.inf
    for step in range(n_step):
        # conduct learning step first
        model.train()
        mg = px_learning_step(R, train_data[:, xid], step = step, total = n_step,
                              adv = adv, wrapper = px_container)
        py_learning_step(step, R, data_loader, model, optimizer, 
                         scheduler, max_epoch = n_epoch)
        model.eval()

        # evaluate the mass of each data
        cnet = ContCNet(mg, py, xid, yid)
        masses = cnet.mass(train_data, logmode = True)

        # conduct adversial step
        if adv:
            R = adversial_step(masses, delta = delta)

        # 07-10-23 22:32, now gurantees to satisfy
        # verify if R satisfy the constrains 
        # sat1 = np.sum(R) <= ( train_data.shape[0] + EPS )
        # sat2 = np.sum( np.log( np.power(R, R) ) ) <= ( train_data.shape[0] * delta + EPS )
        
        # if not (sat1 and sat2):
        #     warnings.warn(f'Constrains not satisfied during adversial step.')
        #     break 
        
        score = cnet.mass(val_data, logmode = True).mean()
        if score > best_score:
            best_model = deepcopy(cnet)
            best_score = score 
    
    # revert to best cnet 
    model = best_model.py.nn
    model.eval()
    model.moveto('cpu')
    
    return best_model, best_score

In [9]:
def compute_neighbor_ll(model, data, seed = 7, num_nb = 500, eps = 0.1):
    '''
    compute the worst and average ll for each sample in the data 
    '''
    N, D = data.shape 
    np.random.seed(seed)
    
    results = [] 
    for x in data:
        pertubation = np.random.rand(num_nb, D)
        pertubation = (pertubation - 0.5) * 2 * eps
        nb_data = x.reshape(1,-1) + pertubation
        nb_ll = model.mass(nb_data, logmode = True)

        # we only care about the worst and average 
        results.append([np.min(nb_ll), np.mean(nb_ll)])

    min_nb_ll, avg_nb_ll = list(zip(*results))
    return min_nb_ll, avg_nb_ll
    

In [10]:
from joblib import Parallel, delayed
import random 


def run_exp(dataset, train_conf, nn_conf, param = None):
    # fix all random seed 
    my_seed = 7
    torch.manual_seed(my_seed)
    random.seed(my_seed)
    np.random.seed(my_seed)
    torch.cuda.manual_seed_all(my_seed)

    if param is not None:
        warnings.warn('Running with fixed hyper-parameter, you should do this only when testing.')

    train_conf.show(ignore = {'model_objs'})
    nn_conf.show()
                
    train_data = dataset.train
    val_data = dataset.valid
    
    xid = nn_conf.xid
    yid = nn_conf.yid
    model_conf = deepcopy(nn_conf)
    structure = create_graph(train_data[:, yid], 
                    max_parents = int(PARENT_RATIO * train_data.shape[1]), 
                    corr_thresh = CORR_THRESH)

    model_conf.num_parents = [len(structure.V[i].parents) for i in range(structure.N)]
    model_conf.structure = structure
    model_conf.device = 'cpu'

    if train_conf.model_objs is None:
        # shared model objects across adversial and non-advesial 
        train_conf.model_objs = [ PGNN(len(xid), model_conf.num_parents, model_conf)
            for i in range(train_conf.n_pretrain)]

    # do hyper selection if param is not specified
    
    if param is None:
        if testing:
            learning_rates = [5e-3, 1e-2]
            weight_decays = [1e-4]
        else:
            learning_rates = [1e-2, 3.3e-3, 1e-3]
            weight_decays = [1e-3, 1e-4]

        all_params = list(product(learning_rates, weight_decays))

        results = Parallel(n_jobs = 1, prefer = 'threads')(
            delayed(train_model)(train_data, val_data, train_conf, model_conf,
                    *comb, cuda_device)
            for comb in all_params
        )

        models, scores = list(zip(*results))
        ind = np.argmax(scores)
        cnet = models[ind]
        print('Best hyper parameter is: {}'.format(all_params[ind]))
    else:
        # print('Running with manual hyper parameters: {}'.format(param))
        cnet = train_model(train_data, val_data, train_conf, model_conf, *param, 'cuda:0')[0]


    avg_stats = []
    for test_X in dataset.test:
        cur_mass = cnet.mass(test_X, logmode = True)
        mass_stat = five_number_statistic(cur_mass)
        logger.write('TestLL p25:{} median:{} p75:{} Mean:{} Std:{}'.format(*mass_stat))
        avg_stats.append( str(mass_stat[3]) )

    for cur_mass, name in zip(compute_neighbor_ll(cnet, dataset.test[0]),
                            ['Worst NB LL','Avg NB LL']):
                                
        mass_stat = five_number_statistic(cur_mass)
        logger.write('{} p25:{} median:{} p75:{} Mean:{} Std:{}'.format(name, *mass_stat))
        avg_stats.append( str(mass_stat[3]) )

    
    res_log.write(','.join(avg_stats), echo = 0)
    return cnet
        

# Experiement of Loglikelihoods

In [11]:
# define some common parameters across experiments
from models.ours.ContCNet import ContCNet
from utmLib.ml.GBN import GBN
from models.ours.NNCondMG import MyObject, create_graph, PGNN, NeuralNetCondMG
from models.ours.Gaussians import MultivariateGaussain

############################################################
np.random.seed(7)

# Gloable variables
G_value_scale = 0.2          # the sscale we mess up the test set
TRAINING_RATIO = 0.85
VALID_RATIO = 0.2
PARENT_RATIO = 0.3           # control the number of parents in the GBN template
CORR_THRESH = 0.05           # similar function as above
EPS = 1e-2
VERBOSE = 1
# px_learning_step = mg_learning_step
px_learning_step = partial(mixmg_learning_step, n_comp = 3)

############################################################

# all dataset meta-data specify
default_options = MyObject()
default_options.root_dir = '/home/leondong/proj/robust/dataset/'
default_options.down_sample = False
default_options.with_label = False
default_options.num_per_class = 1000
default_options.normalize = True
default_options.transform = None

dataset_names = [('mnist', 'mnist/'), ('airquality','airquality/AirQualityUCI.csv'), 
                 ('parkinson', 'parkinson/parkinsons_updrs.data'),
                ('energy','energy/data.csv'), ('hepmass','hepmass/hepmass.h5'),
                ('miniboone', 'miniboone/miniboone.h5'), 
                 ('onlinenews','onlinenews/OnlineNewsPopularity.csv'),
                ('superconduct','superconduct/data.csv'),
                ('sdd', 'SDD/Sensorless_drive_diagnosis.txt')]

loader_options = MyObject()
for name, path in dataset_names: 
    l_op = deepcopy(default_options)
    l_op.data_path = path
    loader_options[name] = l_op

# custom field for some dataset
loader_options.mnist.transform = './output/vae-mnist-e250-d20.pkl'
    

In [12]:
import importlib
from sklearn.preprocessing import StandardScaler
# load the given dataset
cur_options = loader_options[data_name]
loader_module = importlib.import_module('loaders.{}'.format(data_name))
dataset = loader_module.load_data(cur_options)

# train, test split
if len(dataset) == 2:
    train, test = dataset
else:
    np.random.shuffle(dataset)
    n_train = int(dataset.shape[0] * TRAINING_RATIO)
    train = dataset[: n_train]
    test = dataset[n_train:]

# convert to float32 type
train = train.astype('f4')
test = test.astype('f4')

# handle test case 
if testing:
    train = train[:1000]
    test = test[:100]

if cur_options.transform is None:
    # do standardize
    scaler = StandardScaler().fit(train)
    train, test = [scaler.transform(x) for x in [train,test]]
      
# shuffle the train split
assert(len(train.shape) == 2), "Data size does not match"
np.random.shuffle(train)
    
# train, valid split
n_valid = int(train.shape[0] * VALID_RATIO)
valid = train[:n_valid]
train = train[n_valid:]

# random messup the test
test_gaussian = gaussian_noise(test, 1, G_value_scale)
test_pj = pixel_jitter(test, 0.2, -G_value_scale, G_value_scale)

# conduct transformation 
if cur_options.transform is not None:
    model = utils.pkload(cur_options.transform)
    model.model.to('cpu')
    model.device = 'cpu'
    train,valid,test,test_gaussian,test_pj = [model.transform(data) 
                             for data in [train,valid,test,test_gaussian,test_pj]]
    

# wrap up the dataset together
dataset = MyObject()
dataset.train = train
dataset.valid = valid
# dataset.valid = np.vstack([valid, gaussian_noise(valid, 1, G_value_scale)])
dataset.test = [test, test_gaussian, test_pj]

logger.write(f'{data_name} load complete!')

Loading parkinson data .....
parkinson load complete!


In [13]:
# find cutset variables using heuristical algorithm
COND_RATIO = 0.25
MIN_COND_NUM = 1
xid = list( variance_methods( dataset.train , (COND_RATIO, MIN_COND_NUM) ) )
yid = list( np.setdiff1d( np.arange(dataset.train.shape[1]), xid) ) 

############################################################
dim_for_95var = pca_analysis(dataset.train, percentiles = [0.95])[0]
hyper_parameters =  None #  (0.01, 1e-4) 
############################################################
pgnn_conf = MyObject()
pgnn_conf.depth = 3
pgnn_conf.drop_out = 0.0
pgnn_conf.compress_rate = 2
pgnn_conf.prec_thresh = (1e-2, 1e+2)
pgnn_conf.feature_size = dim_for_95var * int( 1+np.log(dataset.train.shape[1]) )
pgnn_conf.max_header_size = len(yid)
pgnn_conf.xid = xid
pgnn_conf.yid = yid

train_conf = MyObject()
train_conf.init_epoch = 50
train_conf.n_step = 150
train_conf.n_epoch = 1
# train_conf.batch_size = 512
train_conf.adversial = True
train_conf.delta = float(G_delta)
train_conf.n_pretrain = 4
train_conf.pre_epochs = 25
train_conf.model_objs = None

# adaptive batch size 
min_batch_size = 64
min_iter_per_epoch = 100
n_train_nums = dataset.train.shape[0]
expect_bs_size = int(n_train_nums / min_iter_per_epoch)
train_conf.batch_size = max(expect_bs_size, min_batch_size)

if testing:
    train_conf.init_epoch = 25
    train_conf.n_step = 25
    train_conf.pre_epochs = 10


In [14]:
# run adversial experiment
cuda_device = 'cuda:{}'.format(os.getpid() % 2)
cnet_adv = run_exp(dataset, train_conf, pgnn_conf, param = hyper_parameters)

init_epoch -> 25
    n_step -> 25
   n_epoch -> 1
 adversial -> True
     delta -> 0.5
n_pretrain -> 4
pre_epochs -> 10
batch_size -> 64
          depth -> 3
       drop_out -> 0.0
  compress_rate -> 2
    prec_thresh -> (0.01, 100.0)
   feature_size -> 21
max_header_size -> 11
            xid -> [11, 13, 7, 14]
            yid -> [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12]


/tmp/ipykernel_25508/717268461.py:10: RuntimeWarning: overflow encountered in power
  return np.log(np.power(w,w)).sum() <= M


/tmp/ipykernel_25508/717268461.py:10: RuntimeWarning: overflow encountered in power
  return np.log(np.power(w,w)).sum() <= M


Best hyper parameter is: (0.01, 0.0001)2224799442291263
TestLL p25:-10.4361 median:-6.7038 p75:-4.3833 Mean:-9.9534 Std:17.0556
TestLL p25:-14.9713 median:-11.4783 p75:-8.6374 Mean:-14.353 Std:16.4938
TestLL p25:-19.7064 median:-11.1181 p75:-7.7839 Mean:-17.7732 Std:23.3438
Worst NB LL p25:-13.6257 median:-9.1123 p75:-6.9068 Mean:-12.9177 Std:18.2826
Avg NB LL p25:-10.7718 median:-7.0435 p75:-4.8906 Mean:-10.3332 Std:17.0049


In [15]:
# non-adversial case
train_conf.adversial = False
# train_conf.n_step, train_conf.n_epoch = train_conf.n_epoch, train_conf.n_step
cnet_std = run_exp(dataset, train_conf, pgnn_conf, param = hyper_parameters)

init_epoch -> 25
    n_step -> 25
   n_epoch -> 1
 adversial -> False
     delta -> 0.5
n_pretrain -> 4
pre_epochs -> 10
batch_size -> 64
          depth -> 3
       drop_out -> 0.0
  compress_rate -> 2
    prec_thresh -> (0.01, 100.0)
   feature_size -> 21
max_header_size -> 11
            xid -> [11, 13, 7, 14]
            yid -> [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12]
Best hyper parameter is: (0.01, 0.0001)2324804592132574834
TestLL p25:-10.1441 median:-6.3774 p75:-3.9273 Mean:-9.4768 Std:16.2435
TestLL p25:-14.7325 median:-11.5488 p75:-8.5247 Mean:-13.8896 Std:15.1321
TestLL p25:-18.3973 median:-11.0145 p75:-8.0415 Mean:-17.1029 Std:19.1755
Worst NB LL p25:-13.0466 median:-9.0963 p75:-6.3424 Mean:-12.4194 Std:17.3611
Avg NB LL p25:-10.462 median:-6.7142 p75:-4.5604 Mean:-9.8554 Std:16.2051


## The following code is used for analysis purpose only

In [16]:
ana_log = Logger(log_file + '.ana', with_time = False)
ori, gau, pj = dataset.test
cnet_adv.mass(ori,logmode= True).mean()

-9.953390432984197

In [17]:
cnet_std.mass(ori,logmode= True).mean()

-9.47678106525377

In [18]:
def mass_ex(self, Z, logmode = True):
    # compute the density of samples, can be single sample as well
    if len(Z.shape) == 1:
        Z = Z.reshape(1, -1)
    
    X = Z[:, self.xids]
    Y = Z[:, self.yids]
    
    massX = self.px.mass(X, logmode = True)
    massY = self.py.mass(Y, X, logmode = True)
    
    return [massX, massY]

mX,mY = mass_ex(cnet_adv, ori)
print(np.mean(mX + mY))
ana_log.write( (five_number_statistic(mX), five_number_statistic(mY)), echo = True )

-9.953390432984197
([-6.1147, -3.8103, -3.0088, -4.6572, 2.2846], [-4.6021, -2.4867, -0.6518, -5.2962, 15.9592])


In [19]:
mX,mY = mass_ex(cnet_std, ori)
ana_log.write( (five_number_statistic(mX), five_number_statistic(mY)), echo = True )

([-5.4827, -3.6317, -2.8554, -4.4188, 2.5392], [-4.7518, -2.5521, -0.7041, -5.058, 14.2557])
